In [9]:
import re
import sqlite3
from pathlib import Path
from datetime import date
import requests
import pandas as pd
from bs4 import BeautifulSoup, Tag
from types_ import JobFilter

In [10]:
FILTER: JobFilter = ['jobType_INTERNSHIP', 'postingDate_7']
CREATE_TABLE_QUERY = Path('../sql/tables.sql').read_text()

In [ ]:
def get_job_listing(
        # row_index: int = 1,
        filters: JobFilter | None = None,
    ) -> BeautifulSoup:
    """This function fetches job listings from the BMW Group website.
    Args:
        filters (JobFilter | None): A list of filters to apply to the job search.
            If None, no filters are applied.
    Returns:
        BeautifulSoup: A BeautifulSoup object containing the listed jobs."""

    url = (
        'https://www.bmwgroup.jobs/'
        'en/_jcr_content/main/layoutcontainer_5337/jobfinder30.jobfinder_table.content.html?'
        f'filterSearch={",".join(filters)}' if filters else ''
    )

    response = requests.get(url)

    html = response.content.decode('utf-8')

    return BeautifulSoup(html, 'html.parser').find_all('div', class_ = 'grp-jobfinder__wrapper')

In [ ]:
def get_job_page(link: str) -> str:
    """Simply fetches the job page content from the given link.
    Args:
        link (str): The URL of the job page.
    Returns:
        str: The HTML content of the job page."""

    return requests.get(link).content.decode('utf-8')

In [ ]:
Html = str

def get_job_description(job_page: Html) -> Html:
    """This function extracts the job description from the job page HTML, but keeps the formatting.
    Args:
        job_page (Html): The HTML content of the job page.
    Returns:
        str: The formatted job description as a html string."""

    description =  BeautifulSoup(job_page, 'html.parser').find('div', class_ = 'container-layout container no-top-spacing no-bottom-spacing')

    # Remove all attributes that are not needed since i just need the formatted HTML.
    for tag in description.find_all(True):
        tag: Tag = tag
        if 'class' in tag.attrs:
            del tag.attrs['class']
        if 'itemprop' in tag.attrs:
            del tag.attrs['itemprop']

    return (
        description.decode_contents()
        .replace('\n', '')
        .replace('\t', '')
        .replace('\r', '')
        .strip()
    )

In [ ]:
def get_job_publishing_date(job_infos: Tag) -> date:
    """Extracts the job publishing date from the job information section.
    Args:
        job_infos (Tag): A BeautifulSoup Tag object containing the job information.
    Returns:
        date: The date when the job was published, as a datetime.date object."""

    date_str = job_infos.find('div', class_ = 'grp-jobfinder__cell-publication').text.strip()
    date_str = re.search(r'\d{2}\.\d{2}\.\d{4}', date_str).group()

    return pd.to_datetime(date_str, format = '%d.%m.%Y').date()

## Extracting jobs data

In [15]:
jobs = list()
for job in get_job_listing(FILTER):
    job: Tag = job

    link = 'https://www.bmwgroup.jobs/' + job.find('a').get('href')
    html = get_job_page(link)

    jobs.append(
        {
            "job_portal_id":    job.get('data-job-id'),
            "job_link":         link,
            "job_title":        job.find('a').get('aria-label'),
            "job_description":  get_job_description(html),
            "job_posting_date": get_job_publishing_date(job),
            "job_type":         job.find('a').find('div', class_='grp-jobfinder-cell-refno').get('data-job-type'),
            "job_field":        job.find('a').find('div', class_='grp-jobfinder-cell-refno').get('data-job-field'),
            "job_city":         job.find('a').find('div', class_='grp-jobfinder-cell-refno').get('data-job-location'),
        }
    )

jobs = pd.DataFrame(jobs)
display(jobs)

,job_portal_id,job_link,job_title,job_description,job_posting_date,job_type,job_field,job_city
0,160441,https://www.bmwgroup.jobs//en/jobfinder/job-de...,Praktikant Social Media und Website (w/m/x),<div><div><p><b>EIN GUTES PRAKTIKUM IST PRAKTI...,2025-06-06,Internship,External Corporate Communication,Munich
1,160337,https://www.bmwgroup.jobs//en/jobfinder/job-de...,Praktikant Versuch Antriebsentwicklung BMW M (...,<div><div><p><b>EIN GUTES PRAKTIKUM IST PRAKTI...,2025-06-06,Internship,Drive Train,Munich
2,160305,https://www.bmwgroup.jobs//en/jobfinder/job-de...,Praktikant Kommunikation Digital Experience (w...,<div><div><p><b>EIN GUTES PRAKTIKUM IST PRAKTI...,2025-06-06,Internship,External Corporate Communication,Munich
3,160283,https://www.bmwgroup.jobs//en/jobfinder/job-de...,Praktikant Social Media Analyse (w/m/x),<div><div><p><b>EIN GUTES PRAKTIKUM IST PRAKTI...,2025-06-06,Internship,External Corporate Communication,Munich
4,160279,https://www.bmwgroup.jobs//en/jobfinder/job-de...,"Praktikant Social Media, Unternehmenskommunika...",<div><div><p><b>EIN GUTES PRAKTIKUM IST PRAKTI...,2025-06-06,Internship,Internal Corporate Communication,Munich
...,...,...,...,...,...,...,...,...
118,157848,https://www.bmwgroup.jobs//en/jobfinder/job-de...,Praktikant Planung Prototypenbau Hochvoltspeic...,<div><div><p><b>EIN GUTES PRAKTIKUM IST PRAKTI...,2025-06-02,Internship,Production planning and control,Munich
119,155818,https://www.bmwgroup.jobs//en/jobfinder/job-de...,Sales Intern,<div><div><p><strong>A GOOD INTERNSHIP IS NEVE...,2025-06-02,Internship,Sales,Budapest
120,152946,https://www.bmwgroup.jobs//en/jobfinder/job-de...,Praktikant Konzernrevision BMW AG (w/m/x),<div><div><p><b>EIN GUTES PRAKTIKUM IST PRAKTI...,2025-06-02,Internship,Audit,Munich
121,141971,https://www.bmwgroup.jobs//en/jobfinder/job-de...,"Rolls Royce Motor Car, Goodwood - Sales Planni...",<div><div><p>A GOOD INTERNSHIP IS NEVER HANDS ...,2025-06-02,Internship,Sales Operations,Goodwood


# Loading to database

In [16]:
with sqlite3.connect('../data/jobs.db') as connection:

    connection.executescript(CREATE_TABLE_QUERY)

    already_exists = pd.read_sql('SELECT job_portal_id FROM jobs', connection)

    jobs = jobs[~jobs['job_portal_id'].isin(already_exists['job_portal_id'])]

    jobs.to_sql(
        name = 'jobs',
        con = connection,
        if_exists = 'append',
        index = False,
    )